# Funciones para encontrar usuarios a testear

In [7]:
import pandas as pd
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse

from tqdm import tqdm

In [2]:
#configuro proxy y headers
url_proxy = 'https://free-proxy-list.net/anonymous-proxy.html'
response_pxy = requests.get(url_proxy)
pxySoup = BeautifulSoup(response_pxy.text, 'html.parser')
rowsPxy = []
headPxy = []
for tablePart in pxySoup.find_all(attrs={'id':'proxylisttable'})[0].find_all(True, recursive=False):
    if(tablePart.name == 'thead'):
        for row in tablePart.find_all(True, recursive=False):
            headPxy = [tr.text for tr in row]
    if(tablePart.name == 'tbody'):
        for row in tablePart.find_all(True, recursive=False):
            rowsPxy.append([tr.text for tr in row])
proxiesDf = pd.DataFrame(data=rowsPxy, columns=headPxy)
proxiesDf = proxiesDf[(proxiesDf['Anonymity']=='elite proxy')].sort_values(by='Https', ascending=False)
headersChrome = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
}
proxies = {
    "http": proxiesDf[proxiesDf.Https=='no'].loc[:,'IP Address'].values[0]
}

In [25]:
headersMore = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Host': 'twitter.com',
    'Accept-Language': 'es-AR,es;q=0.8,en-US;q=0.5,en;q=0.3',
    'Accept-Encoding': 'gzip, deflate, br',
    'X-Twitter-Active-User': 'yes',
    'X-Requested-With': 'XMLHttpRequest',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
}

def getNewPage(term, searchType, maxPosition):
    pageResponse = requests.get(
        'https://twitter.com/i/search/timeline',
        params={
            'vertical': 'news',
            'q':'#'+term,
            'max_position': maxPosition,
            'include_entities': 1,
            'include_available_features': 1,
            'vertical': 'news',
            'reset_error_state': False
        },
        proxies=proxies,
        headers=headersMore
    )
    print(pageResponse.json().keys())

def searchTerm(term, searchType='q'):
    pageResponse = False
    if(searchType == 'q'):
        pageResponse = requests.get('https://twitter.com/search?q='+term, headers=headersChrome, proxies=proxies)
    else:
        pageResponse = requests.get('https://twitter.com/hashtag/'+term, headers=headersChrome, proxies=proxies)
    twData = BeautifulSoup(pageResponse.text, 'html.parser')
    initialStream = twData.findAll(attrs={'id':'timeline'})[0].findAll(attrs={'class':'stream-container'})[0]
    dataMax = initialStream.attrs['data-max-position']
    dataMin = initialStream.attrs['data-min-position']
    streamItems = initialStream.findAll(attrs={'id':'stream-items-id'})[0]
    tweetList = streamItems.findAll('li', recursive=False)
#     for tweet in tqdm(tweetList):
#         print(
#             tweet.findAll(attrs={'class':'username'})[0].text[1:]
#         )
    maxPages = 1
    for pageNum in range(maxPages):
        response = getNewPage(term, searchType, dataMax)
searchTerm('cfk', '#')

dict_keys(['min_position', 'has_more_items', 'items_html', 'new_latent_count', 'focused_refresh_interval'])
